# Notebook 3: Neural networks with Keras

On to the most exciting machine learning technique: Neural networks. as you will see they are just as easy to use as the sklearn methods.

There are several deep learning libraries for Python. The three most commonly used are:

- Keras: High level library based on Tensorflow (or others) that is easy to use and flexible enough for most standard users. It has a great documentation and online support.
- Tensorflow: Google's neural network library. Most widely used in ML research. Flexible and powerful but also (unnecessarily?) complicated.
- Pytorch: The newcomer developed by Facebook. Flexible like Tensorflow but with a nicer, more Pythonic API.

Here we will use Keras which is a great start for most tasks.

In [1]:
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

In [2]:
DATADIR = './dataset1/'

In [3]:
# Copy from previous notebook
def create_sub(preds, fn=None):
    df =  pd.DataFrame({'Id': range(len(preds)), 'Expected': preds})
    if fn is not None: df.to_csv(DATADIR + fn, index=False)
    return df

In [4]:
# Load the preprocessed data
with open('./tmp/preproc_data.pkl', 'rb') as f:
    X_train, y_train, X_valid, y_valid, X_test = pickle.load(f)
with open('./tmp/dfs.pkl', 'rb') as f:
    df_train, df_test = pickle.load(f)

## Linear regression: The Keras way

First, let's build our own linear regression algorithm with stochastic gradient descent. This should give us the same solution as the sklearn linear regression we did in the last notebook.

In [5]:
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import SGD, Adam

Using TensorFlow backend.
/home/s/S.Rasp/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
X_train.shape

(728008, 22)

There are two ways to build a model in Keras. We will start with the easier, a Sequential model. This means that it is a succession of layers. For linear regression we only have one linear layer that maps the inputs to the outputs. Layers where all inputs are connected to all outputs are called fully-connected or Dense layers.

In [7]:
ln = Sequential([Dense(1, input_shape=(22,), activation='linear')])

In [8]:
ln.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 1)                 23        
Total params: 23
Trainable params: 23
Non-trainable params: 0
_________________________________________________________________


We can see that the model has 23 parameters, 22 coefficients plus one bias term.

Next we need to compile the model, which basically means telling Keras which optimizer to use and which loss function to minimize. In the background it also randomly initializes the weights and biases at this stage.

We will use the Adam optimizer, which is a fancy version of SGD: https://arxiv.org/abs/1412.6980

In [9]:
ln.compile(Adam(1e-1), 'mse')

Now we can train/fit the model. For this we specify the training data, the batch size, the number of epochs. Optionally, we can pass on the validation data, so that we get a validation score every epoch.

In [10]:
ln.fit(X_train, y_train, 10_000, epochs=12, validation_data=(X_valid, y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/12
728008/728008 [==============================] - 2s 2us/step - loss: 30.6667 - val_loss: 7.1694
Epoch 2/12
728008/728008 [==============================] - 1s 1us/step - loss: 4.0894 - val_loss: 3.7304
Epoch 3/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.2881 - val_loss: 3.5758
Epoch 4/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.1791 - val_loss: 3.4829
Epoch 5/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.1201 - val_loss: 3.4166
Epoch 6/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0810 - val_loss: 3.3657
Epoch 7/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0544 - val_loss: 3.3249
Epoch 8/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0359 - val_loss: 3.2897
Epoch 9/12
728008/728008 [==============================] - 1s 1us/step - loss: 3.0235 - val_loss: 

In [17]:
from sklearn.metrics import r2_score
def print_scores(m, X_train=X_train, X_valid=X_valid):
    preds = m.predict(X_valid, 10000)
    print('Train R2 = ', r2_score(y_train, m.predict(X_train, 10000)), 
          ', Valid R2 = ', r2_score(y_valid, preds), ', Valid MSE = ', 
          m.evaluate(X_valid, y_valid, 10000, 0))

In [18]:
print_scores(ln)

Train R2 =  0.9318499819537857 , Valid R2 =  0.9184469773803242 , Valid MSE =  3.253879491869268


Recall that with the sklearn linear regression algorithm we got a score of around 3.24. This indicates that we are doing pretty much the same here. But we can do better!

## Neural network with a hidden layer

Now let's actually build a neural network with a hidden layer. For this we simply add another Dense layer but this time with a non-linear activation function. In our case this is a Rectified Linear Unit or relu. There is no set rule for how many hidden layers or nodes to use. For this we just need to employ trial-and-error.

BTW: You will often see people using powers of two for the batch size or the number of nodes. This is for optimization purposes on the GPU, which are not crucial in our case.

In [19]:
nn = Sequential([
    Dense(256, input_shape=(22,), activation='relu'),
    Dense(1, activation='linear')
])

In [20]:
nn.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 256)               5888      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 6,145
Trainable params: 6,145
Non-trainable params: 0
_________________________________________________________________


In [21]:
nn.compile(Adam(1e-3), 'mse')

In [22]:
nn.fit(X_train, y_train, 1024, epochs=12, validation_data=(X_valid, y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/12
728008/728008 [==============================] - 5s 6us/step - loss: 9.1067 - val_loss: 3.3649
Epoch 2/12
728008/728008 [==============================] - 4s 6us/step - loss: 2.8411 - val_loss: 3.0338
Epoch 3/12
728008/728008 [==============================] - 5s 6us/step - loss: 2.6591 - val_loss: 2.9547
Epoch 4/12
728008/728008 [==============================] - 5s 6us/step - loss: 2.5818 - val_loss: 2.9393
Epoch 5/12
728008/728008 [==============================] - 5s 6us/step - loss: 2.5333 - val_loss: 2.9477
Epoch 6/12
728008/728008 [==============================] - 5s 7us/step - loss: 2.4925 - val_loss: 2.9083
Epoch 7/12
728008/728008 [==============================] - 5s 6us/step - loss: 2.4592 - val_loss: 2.9028
Epoch 8/12
728008/728008 [==============================] - 4s 6us/step - loss: 2.4299 - val_loss: 2.8874
Epoch 9/12
728008/728008 [==============================] - 5s 6us/step - loss: 2.4049 - val_loss: 2

In [23]:
print_scores(nn)

Train R2 =  0.9472283991753295 , Valid R2 =  0.9273305186632155 , Valid MSE =  2.899435638055828


*Some notes on the results*

- We see that we are quite heavily overfitting. This makes sense in a model with 6k parameters. There are techniques to prevent overfitting in neural networks, most notably dropout and weight decay (L2 regularization). We will cover those later. For some reason, I found those techniques not to work for this model and dataset. Can you figure out why?
- The score is quite significantly better than our best single random forest model. This suggests that the nonlinear computing power of the neural network is useful.
- This is not yet a deep neural network. DNNs have several hidden layers. Again, I found that for this dataset a DNN didn't perform better. This might be because the nonlinearities are not very strong, but feel free to prove me wrong.

## ADVANCED TECHNIQUE ALERT: Embeddings

We saw in the previous notebook that the station information might be really important. We could train a separate neural network for each station but that would reduce the amount of training data for each individual model and probably lead to stronger overfitting. 

Here is a different method of using categorical variables in neural networks. Don't worry if you don't understand embeddings right away. The concept is easy but it takes a while to wrap your head around it (it did for me anyways). 

First we need to get continuous station IDs.

In [24]:
split_date = '2015-01-01'

In [25]:
stations = df_train.station
stations_train = df_train.station[df_train.time < split_date]
stations_valid = df_train.station[df_train.time >= split_date]
stations_test = df_test.station

In [26]:
unique_stations = pd.concat([df_train.station, df_test.station]).unique()

In [27]:
stat2id = {s: i for i, s in enumerate(unique_stations)}

In [28]:
ids = stations.apply(lambda x: stat2id[x])

In [29]:
ids_train = ids[df_train.time < split_date]
ids_valid = ids[df_train.time >= split_date]
ids_test = stations_test.apply(lambda x: stat2id[x])

Now we will have to use the Keras's Functional API. Check out the official documentation.

We now have two separate inputs. Our regular features and the station ID.

An embedding is a mapping from an integer to a vector of real numbers. In our case the vector has length two. The elements are also called latent features. These are then concatenates with the regular features and passed through one hidden layer as before.

The latent features are updated along with the weights and biases during training and can now represent station-specific information.

In [30]:
from keras.models import Model

In [31]:
features_in = Input(shape=(22,))
id_in = Input(shape=(1,))
emb = Embedding(len(unique_stations), 2)(id_in)
emb = Flatten()(emb)
x = Concatenate()([features_in, emb])
x = Dense(100, activation='relu')(x)
out = Dense(1, activation='linear')(x)
model = Model(inputs=[features_in, id_in], outputs=out)

In [32]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1, 2)         1044        input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 22)           0                                            
__________________________________________________________________________________________________
flatten_1 (Flatten)             (None, 2)            0           embedding_1[0][0]                
__________________________________________________________________________________________________
concatenat

In [33]:
model.compile('adam', 'mse')

In [34]:
model.fit([X_train, ids_train], y_train, 1024, 10, 
          validation_data=([X_valid, ids_valid], y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/10
728008/728008 [==============================] - 5s 7us/step - loss: 10.0235 - val_loss: 3.1622
Epoch 2/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.6514 - val_loss: 2.8208
Epoch 3/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.4462 - val_loss: 2.7294
Epoch 4/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.3384 - val_loss: 2.6312
Epoch 5/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.2406 - val_loss: 2.5662
Epoch 6/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.1757 - val_loss: 2.5334
Epoch 7/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.1374 - val_loss: 2.5007
Epoch 8/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.1095 - val_loss: 2.4706
Epoch 9/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.0867 - val_loss: 

In [35]:
model.fit([X_train, ids_train], y_train, 1024, 10, 
          validation_data=([X_valid, ids_valid], y_valid))

Train on 728008 samples, validate on 180849 samples
Epoch 1/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.0509 - val_loss: 2.4703
Epoch 2/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.0375 - val_loss: 2.4647
Epoch 3/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.0232 - val_loss: 2.4552
Epoch 4/10
728008/728008 [==============================] - 5s 7us/step - loss: 2.0122 - val_loss: 2.4502
Epoch 5/10
728008/728008 [==============================] - 5s 8us/step - loss: 2.0033 - val_loss: 2.4674
Epoch 6/10
728008/728008 [==============================] - 5s 7us/step - loss: 1.9944 - val_loss: 2.4689
Epoch 7/10
728008/728008 [==============================] - 5s 7us/step - loss: 1.9859 - val_loss: 2.4485
Epoch 8/10
728008/728008 [==============================] - 5s 7us/step - loss: 1.9806 - val_loss: 2.4614
Epoch 9/10
728008/728008 [==============================] - 5s 7us/step - loss: 1.9733 - val_loss: 2

In [39]:
print_scores(model, X_train=[X_train, ids_train], X_valid=[X_valid, ids_valid])

Train R2 =  0.9555081131974397 , Valid R2 =  0.938050104761212 , Valid MSE =  2.4717354316843365


In [41]:
# Submit to Kaggle
df_sub = create_sub(model.predict([X_test, ids_test], 10000).squeeze(), 'nn_emb.csv'); df_sub.head()

,Id,Expected
0,0,3.243141
1,1,1.968979
2,2,0.514032
3,3,2.704151
4,4,2.258676


This technique allows us to build a single model that incorporates station information and gives us the best score. Yay!

But can you do better?

## Your turn

1. As we have seen now there are a lot of hyperparameters. Try playing around with them and get the best score. How do the parameters influence the skill?
2. Try an ensemble of techniques. This means training a few models (can be several NNs or some NNs with some RFs) and averaging the predictions. This is also a way to prevent overfitting and might just increase your score ;)